# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
output_data_file = "../WeatherPy/output_data/cities.csv"
city_data_pd = pd.read_csv(output_data_file)
city_data_pd

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)
locations = city_data_pd[['Lat', 'Lng']]
weights = city_data_pd['Humidity']
fig = gmaps.figure(layout={'width':'950px', 'height': '900px'})
heatmap = gmaps.heatmap_layer(locations, weights=weights)
heatmap.point_radius = 20
heatmap.max_intensity = city_data_pd['Humidity'].max()
fig.add_layer(heatmap)
fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
filtered_df = city_data_pd[(city_data_pd['Max Temp'] > 70) 
                  & (city_data_pd['Max Temp'] < 80) 
                  & (city_data_pd['Wind Speed'] < 10) 
                  & (city_data_pd['Cloudiness'] == 0)]


filtered_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = filtered_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df

In [ ]:
or index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving results for: {row['City'].title()}, {row['Country']}")
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # print the response url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

    # convert response to json
    hotel_data = response.json()
    #pprint(hotel_data)
    
    # extract results
    results = hotel_data['results']
          
    try:
          hotel_name = results[0]['name']
          print(f"Closest hotel is {results[0]['name']}.")
          hotel_df.loc[index, 'Hotel Name'] = hotel_name
    except Exception as e:
          print(f"Exception: {str(e)} skipping....")
          # put 'NaN' as hotel name, so that we can drop it later
          hotel_df.loc[index, 'Hotel Name'] = np.nan
    print("----------------------")

In [ ]:
hotel_df.dropna(axis=0, how='any', inplace=True)
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

# Display Map